<a href="https://www.kaggle.com/code/chizkidd/microgpt?scriptVersionId=297786989" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 1. Karparthy's microGPT version
---

In his own words per his [blog](https://karpathy.github.io/):

>_This is a brief guide to my new art project microgpt, a single file of 200 lines of pure Python with no dependencies that trains and inferences a GPT. This file contains the full algorithmic content of what is needed: dataset of documents, tokenizer, autograd engine, a GPT-2-like neural network architecture, the Adam optimizer, training loop, and inference loop. Everything else is just efficiency. I cannot simplify this any further. This script is the culmination of multiple projects (micrograd, makemore, nanogpt, etc.) and a decade-long obsession to simplify LLMs to their bare essentials, and I think it is beautiful 🥹_

- Tweet: https://x.com/karpathy/status/2021694437152157847
- Blog: https://karpathy.github.io/2026/02/12/microgpt/
- Code: https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95

In [2]:
"""
The most atomic way to train and inference a GPT in pure, dependency-free Python.
This file is the complete algorithm.
Everything else is just efficiency.

@karpathy
"""

import os       # os.path.exists
import math     # math.log, math.exp
import random   # random.seed, random.choices, random.gauss, random.shuffle
random.seed(42) # Let there be order among chaos

# Let there be an input dataset `docs`: list[str] of documents (e.g. a dataset of names)
if not os.path.exists('input.txt'):
    import urllib.request
    names_url = 'https://raw.githubusercontent.com/karpathy/makemore/refs/heads/master/names.txt'
    urllib.request.urlretrieve(names_url, 'input.txt')
docs = [l.strip() for l in open('input.txt').read().strip().split('\n') if l.strip()] # list[str] of documents
random.shuffle(docs)
print(f"num docs: {len(docs)}")

# Let there be a Tokenizer to translate strings to discrete symbols and back
uchars = sorted(set(''.join(docs))) # unique characters in the dataset become token ids 0..n-1
BOS = len(uchars) # token id for the special Beginning of Sequence (BOS) token
vocab_size = len(uchars) + 1 # total number of unique tokens, +1 is for BOS
print(f"vocab size: {vocab_size}")

# Let there be Autograd, to recursively apply the chain rule through a computation graph
class Value:
    __slots__ = ('data', 'grad', '_children', '_local_grads') # Python optimization for memory usage

    def __init__(self, data, children=(), local_grads=()):
        self.data = data                # scalar value of this node calculated during forward pass
        self.grad = 0                   # derivative of the loss w.r.t. this node, calculated in backward pass
        self._children = children       # children of this node in the computation graph
        self._local_grads = local_grads # local derivative of this node w.r.t. its children

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return Value(self.data + other.data, (self, other), (1, 1))

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return Value(self.data * other.data, (self, other), (other.data, self.data))

    def __pow__(self, other): return Value(self.data**other, (self,), (other * self.data**(other-1),))
    def log(self): return Value(math.log(self.data), (self,), (1/self.data,))
    def exp(self): return Value(math.exp(self.data), (self,), (math.exp(self.data),))
    def relu(self): return Value(max(0, self.data), (self,), (float(self.data > 0),))
    def __neg__(self): return self * -1
    def __radd__(self, other): return self + other
    def __sub__(self, other): return self + (-other)
    def __rsub__(self, other): return other + (-self)
    def __rmul__(self, other): return self * other
    def __truediv__(self, other): return self * other**-1
    def __rtruediv__(self, other): return other * self**-1

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._children:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1
        for v in reversed(topo):
            for child, local_grad in zip(v._children, v._local_grads):
                child.grad += local_grad * v.grad

# Initialize the parameters, to store the knowledge of the model.
n_embd = 16     # embedding dimension
n_head = 4      # number of attention heads
n_layer = 1     # number of layers
block_size = 16 # maximum sequence length
head_dim = n_embd // n_head # dimension of each head
matrix = lambda nout, nin, std=0.08: [[Value(random.gauss(0, std)) for _ in range(nin)] for _ in range(nout)]
state_dict = {'wte': matrix(vocab_size, n_embd), 'wpe': matrix(block_size, n_embd), 'lm_head': matrix(vocab_size, n_embd)}
for i in range(n_layer):
    state_dict[f'layer{i}.attn_wq'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wk'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wv'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wo'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.mlp_fc1'] = matrix(4 * n_embd, n_embd)
    state_dict[f'layer{i}.mlp_fc2'] = matrix(n_embd, 4 * n_embd)
params = [p for mat in state_dict.values() for row in mat for p in row] # flatten params into a single list[Value]
print(f"num params: {len(params)}")

# Define the model architecture: a stateless function mapping token sequence and parameters to logits over what comes next.
# Follow GPT-2, blessed among the GPTs, with minor differences: layernorm -> rmsnorm, no biases, GeLU -> ReLU
def linear(x, w):
    return [sum(wi * xi for wi, xi in zip(wo, x)) for wo in w]

def softmax(logits):
    max_val = max(val.data for val in logits)
    exps = [(val - max_val).exp() for val in logits]
    total = sum(exps)
    return [e / total for e in exps]

def rmsnorm(x):
    ms = sum(xi * xi for xi in x) / len(x)
    scale = (ms + 1e-5) ** -0.5
    return [xi * scale for xi in x]

def gpt(token_id, pos_id, keys, values):
    tok_emb = state_dict['wte'][token_id] # token embedding
    pos_emb = state_dict['wpe'][pos_id] # position embedding
    x = [t + p for t, p in zip(tok_emb, pos_emb)] # joint token and position embedding
    x = rmsnorm(x)

    for li in range(n_layer):
        # 1) Multi-head attention block
        x_residual = x
        x = rmsnorm(x)
        q = linear(x, state_dict[f'layer{li}.attn_wq'])
        k = linear(x, state_dict[f'layer{li}.attn_wk'])
        v = linear(x, state_dict[f'layer{li}.attn_wv'])
        keys[li].append(k)
        values[li].append(v)
        x_attn = []
        for h in range(n_head):
            hs = h * head_dim
            q_h = q[hs:hs+head_dim]
            k_h = [ki[hs:hs+head_dim] for ki in keys[li]]
            v_h = [vi[hs:hs+head_dim] for vi in values[li]]
            attn_logits = [sum(q_h[j] * k_h[t][j] for j in range(head_dim)) / head_dim**0.5 for t in range(len(k_h))]
            attn_weights = softmax(attn_logits)
            head_out = [sum(attn_weights[t] * v_h[t][j] for t in range(len(v_h))) for j in range(head_dim)]
            x_attn.extend(head_out)
        x = linear(x_attn, state_dict[f'layer{li}.attn_wo'])
        x = [a + b for a, b in zip(x, x_residual)]
        # 2) MLP block
        x_residual = x
        x = rmsnorm(x)
        x = linear(x, state_dict[f'layer{li}.mlp_fc1'])
        x = [xi.relu() for xi in x]
        x = linear(x, state_dict[f'layer{li}.mlp_fc2'])
        x = [a + b for a, b in zip(x, x_residual)]

    logits = linear(x, state_dict['lm_head'])
    return logits

# Let there be Adam, the blessed optimizer and its buffers
learning_rate, beta1, beta2, eps_adam = 0.01, 0.85, 0.99, 1e-8
m = [0.0] * len(params) # first moment buffer
v = [0.0] * len(params) # second moment buffer

import time
train_start = time.time()
# Repeat in sequence
num_steps = 1000 # number of training steps
for step in range(num_steps):

    # Take single document, tokenize it, surround it with BOS special token on both sides
    doc = docs[step % len(docs)]
    tokens = [BOS] + [uchars.index(ch) for ch in doc] + [BOS]
    n = min(block_size, len(tokens) - 1)

    # Forward the token sequence through the model, building up the computation graph all the way to the loss.
    keys, values = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
    losses = []
    for pos_id in range(n):
        token_id, target_id = tokens[pos_id], tokens[pos_id + 1]
        logits = gpt(token_id, pos_id, keys, values)
        probs = softmax(logits)
        loss_t = -probs[target_id].log()
        losses.append(loss_t)
    loss = (1 / n) * sum(losses) # final average loss over the document sequence. May yours be low.

    # Backward the loss, calculating the gradients with respect to all model parameters.
    loss.backward()

    # Adam optimizer update: update the model parameters based on the corresponding gradients.
    lr_t = learning_rate * (1 - step / num_steps) # linear learning rate decay
    for i, p in enumerate(params):
        m[i] = beta1 * m[i] + (1 - beta1) * p.grad
        v[i] = beta2 * v[i] + (1 - beta2) * p.grad ** 2
        m_hat = m[i] / (1 - beta1 ** (step + 1))
        v_hat = v[i] / (1 - beta2 ** (step + 1))
        p.data -= lr_t * m_hat / (v_hat ** 0.5 + eps_adam)
        p.grad = 0

    print(f"step {step+1:4d} / {num_steps:4d} | loss {loss.data:.4f}")

train_end = time.time()

# Inference: may the model babble back to us
temperature = 0.5 # in (0, 1], control the "creativity" of generated text, low to high
print("\n--- inference (new, hallucinated names) ---")
for sample_idx in range(20):
    keys, values = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
    token_id = BOS
    sample = []
    for pos_id in range(block_size):
        logits = gpt(token_id, pos_id, keys, values)
        probs = softmax([l / temperature for l in logits])
        token_id = random.choices(range(vocab_size), weights=[p.data for p in probs])[0]
        if token_id == BOS:
            break
        sample.append(uchars[token_id])
    print(f"sample {sample_idx+1:2d}: {''.join(sample)}")


final_end = time.time()

print(f"train start:  {time.strftime('%H:%M:%S', time.localtime(train_start))}")
print(f"train end:    {time.strftime('%H:%M:%S', time.localtime(train_end))}")
print(f"final end:    {time.strftime('%H:%M:%S', time.localtime(final_end))}")
print(f"train time:   {train_end - train_start:.1f}s  ({(train_end - train_start)/60:.1f} min)")
print(f"total time:   {final_end - train_start:.1f}s  ({(final_end - train_start)/60:.1f} min)")

num docs: 32033
vocab size: 27
num params: 4192
step    1 / 1000 | loss 3.3660
step    2 / 1000 | loss 3.4243
step    3 / 1000 | loss 3.1778
step    4 / 1000 | loss 3.0664
step    5 / 1000 | loss 3.2209
step    6 / 1000 | loss 2.9452
step    7 / 1000 | loss 3.2894
step    8 / 1000 | loss 3.3245
step    9 / 1000 | loss 2.8990
step   10 / 1000 | loss 3.2229
step   11 / 1000 | loss 2.7964
step   12 / 1000 | loss 2.9345
step   13 / 1000 | loss 3.0544
step   14 / 1000 | loss 3.0905
step   15 / 1000 | loss 3.0651
step   16 / 1000 | loss 2.7337
step   17 / 1000 | loss 2.8839
step   18 / 1000 | loss 2.8977
step   19 / 1000 | loss 2.7073
step   20 / 1000 | loss 2.7453
step   21 / 1000 | loss 3.7212
step   22 / 1000 | loss 2.8026
step   23 / 1000 | loss 2.8241
step   24 / 1000 | loss 2.0374
step   25 / 1000 | loss 3.3698
step   26 / 1000 | loss 2.9154
step   27 / 1000 | loss 3.2795
step   28 / 1000 | loss 2.9195
step   29 / 1000 | loss 2.3027
step   30 / 1000 | loss 2.2691
step   31 / 1000 | los

## **total time: approx. 3.4 mins**

# 2. My microGPT version
---

Project can be found on [GitHub](https://github.com/chizkidd/microGPT/blob/main/run_microgpt.py). I made improvements to the [Karpathy version](https://x.com/karpathy/status/2021694437152157847):

1. PyTorch/GPU port with multi-dataset runner
2. Gradient clipping
3. EMA loss smoothing
4. Train/val split (90/10)
5. Top-k sampling
6. Seeded `generate(prompt)`
7. Checkpointing (save/resume)
8. Live loss plots (train raw, EMA, val)
9. 10-line progress logging per dataset
10. Stream vs discrete inference mode
11. Full CLI — all hyperparameters overridable from command line (`--n-embd`, `--lr`, `--beta1`, `--beta2`, `--steps` and more)

>Datasets: names, Pokémon, cities, English words, Paul Graham essays, Shakespeare.



In [8]:
!find /kaggle/input/models/chizkidd -type f

/kaggle/input/models/chizkidd/run-microgpt-v2-py/pytorch/default/1/run_microgpt3.py
/kaggle/input/models/chizkidd/run-microgpt-py/pytorch/default/1/run_microgpt2.py


In [13]:
train_start = time.time()

!cp /kaggle/input/models/chizkidd/run-microgpt-v2-py/pytorch/default/1/run_microgpt3.py /kaggle/working/
!python /kaggle/working/run_microgpt3.py \
    --only names \
    --n-embd 16 \
    --n-head 4 \
    --n-layer 1 \
    --block-size 16 \
    --lr 0.01 \
    --beta1 0.85 \
    --beta2 0.99 \
    --eps 1e-8 \
    --steps 1000


final_end = time.time()

print(f"train start:  {time.strftime('%H:%M:%S', time.localtime(train_start))}")
# print(f"train end:    {time.strftime('%H:%M:%S', time.localtime(train_end))}")
print(f"final end:    {time.strftime('%H:%M:%S', time.localtime(final_end))}")
# print(f"train time:   {train_end - train_start:.1f}s  ({(train_end - train_start)/60:.1f} min)")
print(f"total time:   {final_end - train_start:.1f}s  ({(final_end - train_start)/60:.1f} min)")

running 1 dataset(s): ['names']
cli overrides: {'steps': 1000, 'n_embd': 16, 'n_layer': 1, 'n_head': 4, 'block_size': 16, 'lr': 0.01, 'beta1': 0.85, 'beta2': 0.99, 'eps': 1e-08}

  Dataset : names
  Note    : Baby names -- the original benchmark
  Steps   : 1000  |  n_embd=16  n_layer=1  n_head=4
device: cuda
  [cache] datasets/names.txt
num docs: 32033
train: 28829  |  val: 3204
vocab size: 27
num params: 4,224
  [names] step  100/1000 ( 10%)  loss 3.2984  ema 2.7300  val 2.7650
  [names] step  200/1000 ( 20%)  loss 2.3922  ema 2.5703  val 2.4694
  [names] step  300/1000 ( 30%)  loss 2.4642  ema 2.4706  val 2.6220
  [names] step  400/1000 ( 40%)  loss 2.4347  ema 2.5199  val 2.5021
  [names] step  500/1000 ( 50%)  loss 1.9637  ema 2.4574  val 2.5669
  [names] step  600/1000 ( 60%)  loss 2.4858  ema 2.4661  val 2.4196
  [names] step  700/1000 ( 70%)  loss 2.2220  ema 2.4157  val 2.5920
  [names] step  800/1000 ( 80%)  loss 2.2434  ema 2.3328  val 2.2287
  [names] step  900/1000 ( 90%) 

## **total time: approx. 0.4 mins**

## 2.1. Dataset: Paul Graham essays

In [14]:
import time

start = time.time()

!cp /kaggle/input/models/chizkidd/run-microgpt-v2-py/pytorch/default/1/run_microgpt3.py /kaggle/working/run_microgpt.py
!python /kaggle/working/run_microgpt.py --only paul_graham --steps 100000

end = time.time()

print(f"start:   {time.strftime('%H:%M:%S', time.localtime(start))}")
print(f"end:     {time.strftime('%H:%M:%S', time.localtime(end))}")
print(f"elapsed: {end - start:.1f}s  ({(end - start)/60:.1f} min)")

running 1 dataset(s): ['paul_graham']
cli overrides: {'steps': 100000}

  Dataset : paul_graham
  Note    : Paul Graham essays (~200 txt files) -- distinctive prose style
  Steps   : 100000  |  n_embd=64  n_layer=4  n_head=4
device: cuda
  [download] GitHub file listing: https://api.github.com/repos/sgoel97/essay-datasets/contents/paul_graham_essays/text_data
  [saved]    datasets/paul_graham_essays/_pg_index.json  (218 files)
  loaded 9086 paragraphs from 218 PG essays
num docs: 9086
train: 8177  |  val: 909
vocab size: 103
num params: 218,496
  [paul_graham] step 10000/100000 ( 10%)  loss 1.6247  ema 1.6569  val 1.6670
  [paul_graham] step 20000/100000 ( 20%)  loss 1.3581  ema 1.4516  val 1.6669
  [paul_graham] step 30000/100000 ( 30%)  loss 1.5422  ema 1.4516  val 1.9637
  [paul_graham] step 40000/100000 ( 40%)  loss 1.3268  ema 1.4156  val 1.4407
  [paul_graham] step 50000/100000 ( 50%)  loss 1.3737  ema 1.3804  val 1.4837
  [paul_graham] step 60000/100000 ( 60%)  loss 1.2134  ema 